In [13]:
!python3 -m venv venv
!venv/bin/python -m pip install ipykernel
!venv/bin/python -m ipykernel install --user --name=myenv --display-name "Python (myenv)"
%pip install datasets wandb transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.2/118.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 42.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.2/616.2 kB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.0/841.0 kB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.9/443.9 kB 120.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.4/391.4 kB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [36]:
import torch
import torch.optim as optim
torch.cuda.manual_seed_all(423)

In [42]:
project_name = "hpml-hw2-llm2"

In [3]:
from datasets import load_dataset

dataset = load_dataset("imdb")
print(dataset)

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [4]:
from transformers import DistilBertForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "distilbert-base-uncased"

model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)
model.to(device)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [12]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import torch
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_fn(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN
    )

tokenized = dataset.map(tokenize_fn, batched=True)
tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])



Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [13]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

NUMBER_WORKERS = 4
train_loader = DataLoader(tokenized["train"], batch_size= BATCH_SIZE, shuffle=True, collate_fn=data_collator, num_workers=NUMBER_WORKERS)
test_loader = DataLoader(tokenized["test"], batch_size = BATCH_SIZE, collate_fn=data_collator, num_workers = NUMBER_WORKERS)

## C1

In [14]:
import wandb

wandb.init(project="hpml-hw2-llm", name=f"bs{BATCH_SIZE}_lr{LEARNING_RATE}", group = "Warm Up Experiment")

wandb.config.update({
    "model_name": "distilbert-base-uncased",
    "max_len": MAX_LEN,
    "batch_size": BATCH_SIZE,
    "lr": LEARNING_RATE,
    "optimizer": "AdamW",
    "num_workers": NUMBER_WORKERS,
    "epochs": EPOCHS,
    "compile_mode": False
})

In [15]:
from tqdm import tqdm
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

train_loss = []
train_accuracy = []
test_accuracy = []

for epoch in range(EPOCHS):
  model.train()
  total_loss = 0
  total_correct = 0
  total_samples = 0
  for batch in tqdm(train_loader):
      optimizer.zero_grad()

      inputs = {
          "input_ids": batch["input_ids"].to(device),
          "attention_mask": batch["attention_mask"].to(device),
          "labels": batch["labels"].to(device),
      }

      outputs = model(**inputs)
      loss = outputs.loss
      loss.backward()
      optimizer.step()

      logits = outputs.logits
      preds = torch.argmax(logits, dim = 1)
      labels = batch["labels"].to(device)
      correct = (preds == labels).sum().item()
      total_correct += correct
      total_samples += len(labels)
      total_loss += loss.item()

  avg_loss = total_loss / len(train_loader)
  avg_accuracy = total_correct / total_samples
  train_loss.append(avg_loss)
  train_accuracy.append(avg_accuracy)

  model.eval()
  correct = 0
  total = 0

  with torch.no_grad():
      for batch in test_loader:
          inputs = {
              "input_ids": batch["input_ids"].to(device),
              "attention_mask": batch["attention_mask"].to(device),
          }
          labels = batch["labels"].to(device)

          logits = model(**inputs).logits
          preds = torch.argmax(logits, dim=1)

          correct += (preds == labels).sum().item()
          total += labels.size(0)

  accuracy = correct / total
  test_accuracy.append(accuracy)

  wandb.log({"train/loss": train_loss,
           "train/acc": avg_accuracy,
           "test/acc": accuracy})
  print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {avg_loss:.4f}, Train Accuracy: {avg_accuracy:.4f}, Test Accuracy: {accuracy:.4f}")


  0%|          | 0/782 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(tru

Epoch 1/5, Loss: 0.0365, Train Accuracy: 0.9878, Test Accuracy: 0.8652


  0%|          | 0/782 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(tru

Epoch 2/5, Loss: 0.0248, Train Accuracy: 0.9922, Test Accuracy: 0.8415


  0%|          | 0/782 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(tru

Epoch 3/5, Loss: 0.0195, Train Accuracy: 0.9944, Test Accuracy: 0.8399


  0%|          | 0/782 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(tru

Epoch 4/5, Loss: 0.0201, Train Accuracy: 0.9937, Test Accuracy: 0.8400


  0%|          | 0/782 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(tru

Epoch 5/5, Loss: 0.0181, Train Accuracy: 0.9943, Test Accuracy: 0.8186


In [ ]:
wandb.finish()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))
plt.plot(train_loss, label='Train Loss')
plt.plot(train_accuracy, label='Train Accuracy')
plt.xlabel('Epoch')
# plt.ylabel('Value')
plt.title('Figure F1')
plt.legend()
plt.show()

### Table T1

| Epoch | Train Loss | Train Acc | Test Acc |
|-------|------------|-----------|----------|
|1|0.0317|0.9902|0.8764|
|2|0.0246|0.9920|0.8895|
|3|0.0211|0.9932|0.8765|
|4|0.0178|0.9940|0.8791|
|5|0.0173|0.9947|0.8740|


# C2

In [23]:
wandb.init(project="hpml-hw2-llm", name=f"bs{BATCH_SIZE}_lr{LEARNING_RATE}", group = "Baseline Timing")

wandb.config.update({
    "model_name": "distilbert-base-uncased",
    "max_len": MAX_LEN,
    "batch_size": BATCH_SIZE,
    "lr": LEARNING_RATE,
    "optimizer": "AdamW",
    "num_workers": NUMBER_WORKERS,
    "epochs": EPOCHS,
    "compile_mode": False
})

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


test/acc,█▄▄▄▁
train/acc,▁▆█▇█
test/acc,0.81856
train/acc,0.99428


Exception in thread ChkStopThr:
Traceback (most recent call last):
  File "/venv/main/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/venv/main/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "/venv/main/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/venv/main/lib/python3.12/site-packages/wandb/sdk/wandb_run.py", line 308, in check_stop_status
    self._loop_check_status(
  File "/venv/main/lib/python3.12/site-packages/wandb/sdk/wandb_run.py", line 238, in _loop_check_status
    local_handle = request()
                   ^^^^^^^^^
  File "/venv/main/lib/python3.12/site-packages/wandb/sdk/interface/interface.py", line 992, in deliver_stop_status
    return self._deliver_stop_status(status)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/venv/main/lib/python3.12/site-packages/wandb/sdk/interface/interface_shared.py", line

In [24]:
import time

In [26]:
NUM_WORKERS = 4
train_loader = DataLoader(tokenized["train"], batch_size= BATCH_SIZE, shuffle=True, collate_fn=data_collator, num_workers=NUMBER_WORKERS)
test_loader = DataLoader(tokenized["test"], batch_size = BATCH_SIZE, collate_fn=data_collator, num_workers = NUMBER_WORKERS)

model_name = "distilbert-base-uncased"

model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)
model.to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [27]:
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

train_loss = []
train_accuracy = []
test_accuracy = []
data_loading_time_arr = []
compute_time_arr = []
epoch_time_arr = []

for epoch in range(EPOCHS):
  model.train()
  total_loss = 0
  total_correct = 0
  total_samples = 0
  data_loading_time = 0
  training_compute_time = 0
  total_epoch_time = 0
  start_data_loading = time.time()
  start_epoch_time = time.time()
  for batch in train_loader:
      torch.cuda.synchronize()
      end = time.time()
      data_loading_time +=  end - start_data_loading

      optimizer.zero_grad()

      inputs = {
          "input_ids": batch["input_ids"].to(device),
          "attention_mask": batch["attention_mask"].to(device),
          "labels": batch["labels"].to(device),
      }

      start_compute = time.time()
      outputs = model(**inputs)
      loss = outputs.loss
      loss.backward()
      optimizer.step()

      torch.cuda.synchronize()
      end = time.time()
      training_compute_time += end - start_compute

      logits = outputs.logits
      preds = torch.argmax(logits, dim = 1)
      labels = batch["labels"].to(device)
      correct = (preds == labels).sum().item()
      total_correct += correct
      total_samples += len(labels)
      total_loss += loss.item()

      start_data_loading = time.time()

  end = time.time()
  total_epoch_time = end - start_epoch_time

  avg_loss = total_loss / len(train_loader)
  avg_accuracy = total_correct / total_samples
  train_loss.append(avg_loss)
  train_accuracy.append(avg_accuracy)

  model.eval()
  correct = 0
  total = 0

  with torch.no_grad():
      for batch in test_loader:
          inputs = {
              "input_ids": batch["input_ids"].to(device),
              "attention_mask": batch["attention_mask"].to(device),
          }
          labels = batch["labels"].to(device)

          logits = model(**inputs).logits
          preds = torch.argmax(logits, dim=1)

          correct += (preds == labels).sum().item()
          total += labels.size(0)

  accuracy = correct / total
  test_accuracy.append(accuracy)

  wandb.log({"train/loss": train_loss,
            "train/acc": avg_accuracy,
            "test/acc": accuracy,
             "data-loading time": data_loading_time,
             "compute time": training_compute_time,
             "total epoch time": total_epoch_time})

  data_loading_time_arr.append(data_loading_time)
  compute_time_arr.append(training_compute_time)
  epoch_time_arr.append(total_epoch_time)
  print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {avg_loss:.4f}, Train Accuracy: {avg_accuracy:.4f}, Test Accuracy: {accuracy:.4f}, data_loading time: {data_loading_time} \
          compute time: {training_compute_time} total epoch time: {total_epoch_time}")


Epoch 1/1, Loss: 0.3058, Train Accuracy: 0.8690, Test Accuracy: 0.8821, data_loading time: 3.445314645767212           compute time: 40.95216727256775 total epoch time: 44.711804151535034


In [ ]:
wandb.finish()

# C3

In [52]:
def train():
  with wandb.init(
      group="DataLoader Performance",
      config={
        "model_name": "distilbert-base-uncased",
        "max_len": 256, 
        "optimizer": "AdamW", 
        "batch_size": 32,
        "learning_rate": 1e-4,
        "epochs": 5, 
        "compile_mode": False
    }) as run:
      config = wandb.config
      run.name = f"lr{wandb.config.learning_rate}_bs{wandb.config.batch_size}"
      
      train_loader = DataLoader(tokenized["train"], batch_size=config.batch_size, shuffle=True, collate_fn=data_collator, num_workers=config.num_workers)
      test_loader = DataLoader(tokenized["test"], batch_size=config.batch_size, collate_fn=data_collator, num_workers=config.num_workers)
      model_name = "distilbert-base-uncased"
      model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)
      model.to(device)
    
      optimizer = getattr(optim, config.optimizer)(model.parameters(), lr=config.learning_rate)
      print(f"---------start train: batch_size({config.batch_size}) lr({config.learning_rate}) num_workers({config.num_workers}) optimizer({config.optimizer})-----")
      train_loss = []
      train_accuracy = []
      test_accuracy = []
      data_loading_time_arr = []
      compute_time_arr = []
      epoch_time_arr = []
    
      for epoch in range(config.epochs):
        model.train()
        total_loss = 0
        total_correct = 0
        total_samples = 0
        data_loading_time = 0
        training_compute_time = 0
        total_epoch_time = 0
        start_data_loading = time.time()
        start_epoch_time = time.time()
        for batch in train_loader:
            inputs = {
                "input_ids": batch["input_ids"].to(device),
                "attention_mask": batch["attention_mask"].to(device),
                "labels": batch["labels"].to(device),
            }
            torch.cuda.synchronize()
            end = time.time()
            data_loading_time +=  end - start_data_loading

            start_compute = time.time()
            optimizer.zero_grad()
            outputs = model(**inputs)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
    
            torch.cuda.synchronize()
            end = time.time()
            training_compute_time += end - start_compute
    
            logits = outputs.logits
            preds = torch.argmax(logits, dim = 1)
            labels = batch["labels"].to(device)
            correct = (preds == labels).sum().item()
            total_correct += correct
            total_samples += len(labels)
            total_loss += loss.item()
    
            start_data_loading = time.time()
    
        end = time.time()
        total_epoch_time = end - start_epoch_time
    
        avg_loss = total_loss / len(train_loader)
        avg_accuracy = total_correct / total_samples
        train_loss.append(avg_loss)
        train_accuracy.append(avg_accuracy)
    
        model.eval()
        correct = 0
        total = 0
    
        with torch.no_grad():
            for batch in test_loader:
                inputs = {
                    "input_ids": batch["input_ids"].to(device),
                    "attention_mask": batch["attention_mask"].to(device),
                }
                labels = batch["labels"].to(device)
    
                logits = model(**inputs).logits
                preds = torch.argmax(logits, dim=1)
    
                correct += (preds == labels).sum().item()
                total += labels.size(0)
    
        accuracy = correct / total
        test_accuracy.append(accuracy)
    
        wandb.log({"train/loss": train_loss,
                  "train/acc": avg_accuracy,
                  "test/acc": accuracy,
                  "data-loading time": data_loading_time,
                  "compute time": training_compute_time,
                  "total epoch time": total_epoch_time})
    
        data_loading_time_arr.append(data_loading_time)
        compute_time_arr.append(training_compute_time)
        epoch_time_arr.append(total_epoch_time)
        print(f"Epoch {epoch+1}/{config.epochs}, Loss: {avg_loss:.4f}, Train Accuracy: {avg_accuracy:.4f}, Test Accuracy: {accuracy:.4f}, data_loading time: {data_loading_time} \
                compute time: {training_compute_time} total epoch time: {total_epoch_time}")
        print(f"-------------------------------------------------------------------------------")

In [ ]:
sweep_config = {
    'method': 'grid',
    'parameters': {
        'num_workers': {'values': [0, 2, 4, 8]}
    }
}

sweep_id = wandb.sweep(sweep_config, project=project_name)

wandb.agent(sweep_id, function=train, count=4)

Create sweep with ID: lbv42exo
Sweep URL: https://wandb.ai/kaimao-columbia-university/hpml-hw2-llm2/sweeps/lbv42exo


wandb: Agent Starting Run: raa2hle7 with config:
wandb: 	num_workers: 0
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


---------start train: batch_size(32) lr(0.0001) num_workers(0) optimizer(AdamW)-----
Epoch 1/5, Loss: 0.2978, Train Accuracy: 0.8769, Test Accuracy: 0.8896, data_loading time: 3.5919899940490723                 compute time: 41.070215940475464 total epoch time: 44.86136269569397
-------------------------------------------------------------------------------
Epoch 2/5, Loss: 0.1656, Train Accuracy: 0.9385, Test Accuracy: 0.8901, data_loading time: 3.472041368484497                 compute time: 41.388070821762085 total epoch time: 45.00368905067444
-------------------------------------------------------------------------------
Epoch 3/5, Loss: 0.0879, Train Accuracy: 0.9703, Test Accuracy: 0.8908, data_loading time: 3.617926836013794                 compute time: 41.62524342536926 total epoch time: 45.40958905220032
-------------------------------------------------------------------------------
Epoch 4/5, Loss: 0.0607, Train Accuracy: 0.9797, Test Accuracy: 0.8755, data_loading time: 3.

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


...

Epoch 5/5, Loss: 0.0380, Train Accuracy: 0.9870, Test Accuracy: 0.8752, data_loading time: 7.164135456085205                 compute time: 41.64370083808899 total epoch time: 57.40517330169678
-------------------------------------------------------------------------------


compute time,██▄▁▂
data-loading time,▁█▆▅▆
test/acc,█▅▅▁▁
total epoch time,▂██▁▇
train/acc,▁▅▇██
compute time,41.6437
data-loading time,7.16414
test/acc,0.8752
total epoch time,57.40517
train/acc,0.98696


wandb: Agent Starting Run: ae71dmgx with config:
wandb: 	num_workers: 8
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


---------start train: batch_size(32) lr(0.0001) num_workers(8) optimizer(AdamW)-----


# C4

In [22]:
from tqdm import tqdm
from torch.optim import AdamW
import torch.profiler as profiler

EPOCHS = 1
NUMBER_WORKERS = 0
train_loader = DataLoader(tokenized["train"], batch_size= BATCH_SIZE, shuffle=True, collate_fn=data_collator, num_workers=NUMBER_WORKERS)
test_loader = DataLoader(tokenized["test"], batch_size = BATCH_SIZE, collate_fn=data_collator, num_workers = NUMBER_WORKERS)

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

model_name = "distilbert-base-uncased"
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)
model.to(device)

train_loss = []
train_accuracy = []
test_accuracy = []

with profiler.profile(
    activities=[profiler.ProfilerActivity.CPU, profiler.ProfilerActivity.CUDA],
    record_shapes=True,
    profile_memory=True,
    with_stack=True
) as prof:
    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0
        total_correct = 0
        total_samples = 0
        for batch in tqdm(train_loader):
            optimizer.zero_grad()

            with profiler.record_function("data_loading"):
                inputs = {
                    "input_ids": batch["input_ids"].to(device),
                    "attention_mask": batch["attention_mask"].to(device),
                    "labels": batch["labels"].to(device),
                }

            with profiler.record_function("forward"):
                outputs = model(**inputs)

            with profiler.record_function("backward"):
                loss = outputs.loss
                loss.backward()

            with profiler.record_function("optimizer"):
                optimizer.step()

            logits = outputs.logits
            preds = torch.argmax(logits, dim = 1)
            labels = batch["labels"].to(device)
            correct = (preds == labels).sum().item()
            total_correct += correct
            total_samples += len(labels)
            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        avg_accuracy = total_correct / total_samples
        train_loss.append(avg_loss)
        train_accuracy.append(avg_accuracy)

        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for batch in test_loader:
                inputs = {
                    "input_ids": batch["input_ids"].to(device),
                    "attention_mask": batch["attention_mask"].to(device),
                }
                labels = batch["labels"].to(device)

                logits = model(**inputs).logits
                preds = torch.argmax(logits, dim=1)

                correct += (preds == labels).sum().item()
                total += labels.size(0)

        accuracy = correct / total
        test_accuracy.append(accuracy)

        print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {avg_loss:.4f}, Train Accuracy: {avg_accuracy:.4f}, Test Accuracy: {accuracy:.4f}")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 782/782 [00:49<00:00, 15.71it/s]


Epoch 1/1, Loss: 0.6935, Train Accuracy: 0.5051, Test Accuracy: 0.4968


In [ ]:
print("\n===== CPU Profiling =====")

print(prof)

print(prof.key_averages().table(
    sort_by="self_cpu_time_total",  # or "cpu_time_total"
    row_limit=20
))

# ✅ Print GPU-focused results
print("\n===== GPU Profiling =====")
print(prof.key_averages().table(
    sort_by="self_cuda_time_total",  # or "cuda_time_total"
    row_limit=20
))

In [ ]:
# C6

In [43]:
sweep_config = {
    'method': 'grid',
    'parameters': {
        'batch_size': {'values': [16, 32, 64]},
        'learning_rate': {'values': [5e-5, 1e-4, 5e-4]}
    }
}


In [38]:
def train():
  with wandb.init(config={
        "model_name": "distilbert-base-uncased",
        "max_len": 256, 
        "optimizer": "AdamW", 
        "num_workers": 2,
        "epochs": 2, 
        "compile_mode": False
    }) as run:
        
      train_loader = DataLoader(tokenized["train"], batch_size=config.batch_size, shuffle=True, collate_fn=data_collator, num_workers=config.num_workers)
      test_loader = DataLoader(tokenized["test"], batch_size=config.batch_size, collate_fn=data_collator, num_workers=config.num_workers)
      model_name = "distilbert-base-uncased"
      model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)
      model.to(device)
    
      optimizer = getattr(optim, config.optimizer.capitalize())(model.parameters(), lr=config.learning_rate)
      print(f"---------start train: batch_size({config.batch_size}) lr({config.learning_rate}) num_workers({config.num_workers}) optimizer({config.optimizer})-----")
      train_loss = []
      train_accuracy = []
      test_accuracy = []
      data_loading_time_arr = []
      compute_time_arr = []
      epoch_time_arr = []
    
      for epoch in range(config.epochs):
        model.train()
        total_loss = 0
        total_correct = 0
        total_samples = 0
        data_loading_time = 0
        training_compute_time = 0
        total_epoch_time = 0
        start_data_loading = time.time()
        start_epoch_time = time.time()
        for batch in train_loader:
            inputs = {
                "input_ids": batch["input_ids"].to(device),
                "attention_mask": batch["attention_mask"].to(device),
                "labels": batch["labels"].to(device),
            }
            torch.cuda.synchronize()
            end = time.time()
            data_loading_time +=  end - start_data_loading

            start_compute = time.time()
            optimizer.zero_grad()
            outputs = model(**inputs)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
    
            torch.cuda.synchronize()
            end = time.time()
            training_compute_time += end - start_compute
    
            logits = outputs.logits
            preds = torch.argmax(logits, dim = 1)
            labels = batch["labels"].to(device)
            correct = (preds == labels).sum().item()
            total_correct += correct
            total_samples += len(labels)
            total_loss += loss.item()
    
            start_data_loading = time.time()
    
        end = time.time()
        total_epoch_time = end - start_epoch_time
    
        avg_loss = total_loss / len(train_loader)
        avg_accuracy = total_correct / total_samples
        train_loss.append(avg_loss)
        train_accuracy.append(avg_accuracy)
    
        model.eval()
        correct = 0
        total = 0
    
        with torch.no_grad():
            for batch in test_loader:
                inputs = {
                    "input_ids": batch["input_ids"].to(device),
                    "attention_mask": batch["attention_mask"].to(device),
                }
                labels = batch["labels"].to(device)
    
                logits = model(**inputs).logits
                preds = torch.argmax(logits, dim=1)
    
                correct += (preds == labels).sum().item()
                total += labels.size(0)
    
        accuracy = correct / total
        test_accuracy.append(accuracy)
    
        wandb.log({"train/loss": train_loss,
                  "train/acc": avg_accuracy,
                  "test/acc": accuracy,
                  "data-loading time": data_loading_time,
                  "compute time": training_compute_time,
                  "total epoch time": total_epoch_time})
    
        data_loading_time_arr.append(data_loading_time)
        compute_time_arr.append(training_compute_time)
        epoch_time_arr.append(total_epoch_time)
        print(f"Epoch {epoch+1}/{config.epochs}, Loss: {avg_loss:.4f}, Train Accuracy: {avg_accuracy:.4f}, Test Accuracy: {accuracy:.4f}, data_loading time: {data_loading_time} \
                compute time: {training_compute_time} total epoch time: {total_epoch_time}")
        print(f"-------------------------------------------------------------------------------")

In [39]:

sweep_id = wandb.sweep(sweep_config, project=project_name, group="Hyperparameter Sensitivity")

wandb.agent(sweep_id, function=train, count=9)

Create sweep with ID: y2g0jplp
Sweep URL: https://wandb.ai/kaimao-columbia-university/Hyperparameter%20Sensitivity/sweeps/y2g0jplp


wandb: Agent Starting Run: if0a1s74 with config:
wandb: 	batch_size: 16
wandb: 	learning_rate: 5e-05
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


---------start train: batch_size(16) lr(5e-05) num_workers(2) optimizer(AdamW)-----
Epoch 1/2, Loss: 0.2927, Train Accuracy: 0.8786, Test Accuracy: 0.8994, data_loading time: 5.044431447982788                 compute time: 46.83597779273987 total epoch time: 56.109216928482056
-------------------------------------------------------------------------------


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


...

compute time,▁█
data-loading time,▁█
test/acc,▁█
total epoch time,▁█
train/acc,▁█
compute time,40.06726
data-loading time,3.70154
test/acc,0.90516
total epoch time,47.81323
train/acc,0.93996


wandb: Agent Starting Run: 9fm34824 with config:
wandb: 	batch_size: 64
wandb: 	learning_rate: 0.0005
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


---------start train: batch_size(64) lr(0.0005) num_workers(2) optimizer(AdamW)-----
Epoch 1/2, Loss: 0.6971, Train Accuracy: 0.5002, Test Accuracy: 0.5000, data_loading time: 3.5461485385894775                 compute time: 39.772443771362305 total epoch time: 47.27660632133484
-------------------------------------------------------------------------------


wandb: Ctrl + C detected. Stopping sweep.
Traceback (most recent call last):
  File "/tmp/ipykernel_1984/2333432887.py", line 36, in train
    for batch in train_loader:
                 ^^^^^^^^^^^^
  File "/venv/main/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 734, in __next__
    data = self._next_data()
           ^^^^^^^^^^^^^^^^^
  File "/venv/main/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1492, in _next_data
    idx, data = self._get_data()
                ^^^^^^^^^^^^^^^^
  File "/venv/main/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1454, in _get_data
    success, data = self._try_get_data()
                    ^^^^^^^^^^^^^^^^^^^^
  File "/venv/main/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1285, in _try_get_data
    data = self._data_queue.get(timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/venv/main/lib/python3.12/multiprocessing/queues.py", line 122, i